In [11]:
import logging
from clickhouse_driver import Client
from dotenv import load_dotenv
import os
from datetime import datetime, timedelta, date

# Загрузка переменных окружения
load_dotenv()

# Настройка логирования
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("update_balances.log", encoding='utf-8'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger()

# Параметры подключения
clickhouse_host = os.getenv('CLICKHOUSE_HOST', '10.95.19.132')
clickhouse_user = os.getenv('CLICKHOUSE_USER', 'default')
clickhouse_password = os.getenv('CLICKHOUSE_PASSWORD', 'quie1ahpoo5Su0wohpaedae8keeph6bi')
database_name = os.getenv('CLICKHOUSE_DB', 'default')

# Создание клиента ClickHouse
client = Client(
    host=clickhouse_host,
    user=clickhouse_user,
    password=clickhouse_password,
    port=9000,
    secure=False,
    settings={'strings_encoding': 'utf-8'}
)


def calculate_and_update_balances():
    """
    Рассчитывает и обновляет balance_mi8t, balance_mi17, balance_total, 
    stock_mi8t, stock_mi17, stock_total для target_date = initial_date + 1 день
    в таблице OlapCube_VNV.
    """
    try:
        # Извлечь initial_date (самая ранняя дата)
        query_initial_date = f"""
        SELECT MIN(Dates) AS initial_date
        FROM {database_name}.OlapCube_VNV
        """
        initial_date_result = client.execute(query_initial_date)
        initial_date = initial_date_result[0][0]
        logger.info(f"Initial date in OlapCube_VNV: {initial_date}")

        # Проверка наличия initial_date
        if not initial_date:
            logger.error("Не удалось получить initial_date. Завершаем выполнение.")
            return

        # Вычислить target_date = initial_date + 1 день
        if isinstance(initial_date, datetime):
            initial_date_date = initial_date.date()
        elif isinstance(initial_date, date):
            initial_date_date = initial_date
        elif isinstance(initial_date, str):
            try:
                initial_date_date = datetime.strptime(initial_date, "%Y-%m-%d").date()
            except ValueError:
                logger.error(f"Неправильный формат initial_date: {initial_date}")
                return
        else:
            logger.error(f"Неизвестный формат initial_date: {initial_date}")
            return

        target_date = initial_date_date + timedelta(days=1)
        logger.info(f"Target date: {target_date}")

        # 1. Рассчитать balance_mi8t
        query_balance_mi8t = f"""
        SELECT COUNT(*) 
        FROM {database_name}.OlapCube_VNV
        WHERE ac_typ = 'Ми-8Т' AND Status_P = 'Эксплуатация' AND Dates = '{target_date}'
        """
        balance_mi8t_count = client.execute(query_balance_mi8t)[0][0]
        logger.info(f"balance_mi8t_count: {balance_mi8t_count}")

        # Получить mi8t_count
        query_mi8t_count = f"""
        SELECT mi8t_count 
        FROM {database_name}.OlapCube_VNV
        WHERE Dates = '{target_date}'
        LIMIT 1
        """
        mi8t_count_result = client.execute(query_mi8t_count)
        if mi8t_count_result:
            mi8t_count = mi8t_count_result[0][0]
            balance_mi8t = balance_mi8t_count - mi8t_count
            logger.info(f"balance_mi8t: {balance_mi8t} (balance_mi8t_count: {balance_mi8t_count} - mi8t_count: {mi8t_count})")
        else:
            logger.warning(f"mi8t_count не найден для даты {target_date}. Устанавливаем balance_mi8t = balance_mi8t_count.")
            balance_mi8t = balance_mi8t_count

        # 2. Рассчитать balance_mi17
        query_balance_mi17 = f"""
        SELECT COUNT(*) 
        FROM {database_name}.OlapCube_VNV
        WHERE ac_typ = 'Ми-17' AND Status_P = 'Эксплуатация' AND Dates = '{target_date}'
        """
        balance_mi17_count = client.execute(query_balance_mi17)[0][0]
        logger.info(f"balance_mi17_count: {balance_mi17_count}")

        # Получить mi17_count
        query_mi17_count = f"""
        SELECT mi17_count 
        FROM {database_name}.OlapCube_VNV
        WHERE Dates = '{target_date}'
        LIMIT 1
        """
        mi17_count_result = client.execute(query_mi17_count)
        if mi17_count_result:
            mi17_count = mi17_count_result[0][0]
            balance_mi17 = balance_mi17_count - mi17_count
            logger.info(f"balance_mi17: {balance_mi17} (balance_mi17_count: {balance_mi17_count} - mi17_count: {mi17_count})")
        else:
            logger.warning(f"mi17_count не найден для даты {target_date}. Устанавливаем balance_mi17 = balance_mi17_count.")
            balance_mi17 = balance_mi17_count

        # 3. Рассчитать balance_total
        balance_total = balance_mi8t + balance_mi17
        logger.info(f"balance_total: {balance_total} (balance_mi8t: {balance_mi8t} + balance_mi17: {balance_mi17})")

        # 4. Рассчитать stock_mi8t
        query_stock_mi8t = f"""
        SELECT COUNT(*) 
        FROM {database_name}.OlapCube_VNV
        WHERE ac_typ = 'Ми-8Т' AND Status_P = 'Исправен' AND Dates = '{target_date}'
        """
        stock_mi8t = client.execute(query_stock_mi8t)[0][0]
        logger.info(f"stock_mi8t: {stock_mi8t}")

        # 5. Рассчитать stock_mi17
        query_stock_mi17 = f"""
        SELECT COUNT(*) 
        FROM {database_name}.OlapCube_VNV
        WHERE ac_typ = 'Ми-17' AND Status_P = 'Исправен' AND Dates = '{target_date}'
        """
        stock_mi17 = client.execute(query_stock_mi17)[0][0]
        logger.info(f"stock_mi17: {stock_mi17}")

        # 6. Рассчитать stock_empty (ac_typ пусто и Status_P = 'Исправен')
        query_stock_empty = f"""
        SELECT COUNT(*) 
        FROM {database_name}.OlapCube_VNV
        WHERE (ac_typ = '' OR ac_typ IS NULL) AND Status_P = 'Исправен' AND Dates = '{target_date}'
        """
        stock_empty = client.execute(query_stock_empty)[0][0]
        logger.info(f"stock_empty: {stock_empty}")

        # 7. Рассчитать stock_total
        stock_total = stock_mi8t + stock_mi17 + stock_empty
        logger.info(f"stock_total: {stock_total} (stock_mi8t: {stock_mi8t} + stock_mi17: {stock_mi17} + stock_empty: {stock_empty})")

        # 8. Обновить таблицу OlapCube_VNV
        update_query = f"""
        ALTER TABLE {database_name}.OlapCube_VNV 
        UPDATE 
            balance_mi8t = {balance_mi8t},
            balance_mi17 = {balance_mi17},
            balance_total = {balance_total},
            stock_mi8t = {stock_mi8t},
            stock_mi17 = {stock_mi17},
            stock_total = {stock_total}
        WHERE Dates = '{target_date}'
        """
        client.execute(update_query)
        logger.info(f"Баланс и запасы успешно обновлены для даты {target_date}.")

    except Exception as e:
        logger.error(f"Ошибка при расчете или обновлении балансов: {e}", exc_info=True)


def main():
    calculate_and_update_balances()


if __name__ == "__main__":
    main()


2024-12-02 11:05:26,711 - INFO - Initial date in OlapCube_VNV: 2024-11-25
2024-12-02 11:05:26,712 - INFO - Target date: 2024-11-26
2024-12-02 11:05:26,750 - INFO - balance_mi8t_count: 73
2024-12-02 11:05:26,764 - INFO - balance_mi8t: -13.0 (balance_mi8t_count: 73 - mi8t_count: 86.0)
2024-12-02 11:05:26,798 - INFO - balance_mi17_count: 83
2024-12-02 11:05:26,809 - INFO - balance_mi17: 13.0 (balance_mi17_count: 83 - mi17_count: 70.0)
2024-12-02 11:05:26,810 - INFO - balance_total: 0.0 (balance_mi8t: -13.0 + balance_mi17: 13.0)
2024-12-02 11:05:26,833 - INFO - stock_mi8t: 8
2024-12-02 11:05:26,857 - INFO - stock_mi17: 11
2024-12-02 11:05:26,882 - INFO - stock_empty: 1
2024-12-02 11:05:26,883 - INFO - stock_total: 20 (stock_mi8t: 8 + stock_mi17: 11 + stock_empty: 1)
2024-12-02 11:05:26,890 - INFO - Баланс и запасы успешно обновлены для даты 2024-11-26.
